### Урок 2.

In [1]:
import pandas as pd

Наши новости

In [5]:
news = pd.read_csv(r"E:\MLinBusiness\articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [8]:
users = pd.read_csv(r"E:\MLinBusiness\users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [9]:
#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [13]:
#предобработка текстов
import re
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sepa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [18]:
with open(r"E:\MLinBusiness\stopwords.txt") as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

2026

In [19]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [21]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

Wall time: 25.4 s


In [22]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 5min 19s


А теперь в 3 строчки обучим нашу модель

In [23]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Что такое common_dictionary и как он выглядит

In [24]:
common_dictionary[10]

'ватутин'

Все просто - это словарь наших слов

Запускаем обучение

In [25]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 34.3 s


In [26]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [27]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(0, 0.033997413),
 (4, 0.022103788),
 (12, 0.5648602),
 (13, 0.18266107),
 (16, 0.01924703),
 (23, 0.1601219)]

In [28]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: военный россия год nn сша фестиваль белоруссия
topic_1: сша рак американский офицер вероятно риск миллиард
topic_2: млн год рубль тыс цена погибнуть составить
topic_3: год компания который это мочь экономика мозг
topic_4: газ год египет конкурс болезнь умереть иран
topic_5: ракета смерть млрд медведев ступень год девочка
topic_6: дело сотрудник суд который орган уголовный закон
topic_7: вуз лекарство прекращение палата ливия врач регистрация
topic_8: это год который россия российский мочь страна
topic_9: nn наука научный место земля статья университет
topic_10: рынок компания снижение земля это поток продукция
topic_11: год который nn технология новый страна москва
topic_12: это год который человек всё весь свой
topic_13: год который это россия nn российский свой
topic_14: украина украинский гражданин миссия страна киев это
topic_15: таиланд су грузовик пистолет тверской взорваться выстрел
topic_16: статья рейс разместить ск ст приговор смерть
topic_17: район пострадать авария

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [29]:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [30]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.028583,0.0,0.000000,0.0,0.000000,0.0,0.109407,0.0,0.305425,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.044072,0.0
1,4896,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.059823,0.0,0.000000,...,0.0,0.000000,0.0,0.644301,0.0,0.0,0.0,0.0,0.273868,0.0
2,4897,0.033899,0.0,0.000000,0.0,0.022107,0.0,0.000000,0.0,0.000000,...,0.0,0.019271,0.0,0.000000,0.0,0.0,0.0,0.0,0.160127,0.0
3,4898,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.049483,0.0,0.000000,...,0.0,0.000000,0.0,0.104374,0.0,0.0,0.0,0.0,0.094413,0.0
4,4899,0.000000,0.0,0.031404,0.0,0.168765,0.0,0.224456,0.0,0.000000,...,0.0,0.441149,0.0,0.000000,0.0,0.0,0.0,0.0,0.111856,0.0


### Следующий шаг - векторные представления пользователей

In [31]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [32]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [33]:
doc_dict[293622]

array([0.        , 0.        , 0.01870286, 0.        , 0.        ,
       0.        , 0.05289999, 0.        , 0.        , 0.22198573,
       0.        , 0.16042693, 0.17517816, 0.10746825, 0.04548031,
       0.        , 0.        , 0.        , 0.        , 0.20754103,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [34]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding(user_articles_list, vector="mean"):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    if vector=="mean":
        user_vector = np.mean(user_vector, 0)
    elif vector=="mediana":
        user_vector = np.median(user_vector, 0)
    elif vector=="max":
        user_vector = np.max(user_vector, 0)
    return user_vector

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [36]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [38]:
target = pd.read_csv(r"E:\MLinBusiness\users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [39]:
vectors = ['mean', 'mediana', 'max']
logreg = LogisticRegression()

In [40]:
for vector in vectors: 
    
    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, vector=vector), 1)])
    user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
    
    X = pd.merge(user_embeddings, target, 'left')
    X.head(3)
    
    X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=42)
    logreg.fit(X_train, y_train)
    preds = logreg.predict_proba(X_test)[:, 1]
    roc_auc_score(y_test, preds)

   
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    ix = np.argmax(fscore)

    print(f'If vector={vector}: F-Score={fscore[ix]}, Precision={precision[ix]}, Recall={recall[ix]}, Roc auc score={roc_auc_score(y_test, preds)}')
    
    print(f'{vector} ok')
    

If vector=mean: F-Score=0.7184115523465704, Precision=0.6611295681063123, Recall=0.7865612648221344, Roc auc score=0.9585760796034308
mean ok
If vector=mediana: F-Score=0.7357142857142857, Precision=0.6710097719869706, Recall=0.8142292490118577, Roc auc score=0.965184811455437
mediana ok
If vector=max: F-Score=0.8221797323135756, Precision=0.7962962962962963, Recall=0.849802371541502, Roc auc score=0.9847576081865921
max ok
